# BiLSTM Model – CO₂ Prediction with GroupKFold CV

This notebook implements a Bidirectional LSTM (BiLSTM) deep learning model to predict surface seawater CO₂ concentrations (μatm) using multivariate time series data. The workflow includes preprocessing, windowing, GroupKFold cross-validation, feature importance (permutation method), and prediction export.

## Highlights of this version:

• **Input data:** 'Deception_2025_CO2_ocean_meteo_seismic.csv' should include oceanographic (seawater temperature anda salinity), meteorological (solar radiation, air temperature and wind speed), geosphysical (tidal heigh, seismic events), and spatiio-temporal variables (Date, latitude and longitude).

• **Grouping:** Based on latitude bins to account for spatial structure during GroupKFold.

• **Model:** BiLSTM architecture trained per fold with early stopping.

• **Evaluation metrics:** RMSE and R² computed for each fold.

• **Feature importance:** Permutation-based increase in RMSE for each input variable.

• **Best fold selection:** Identified based on lowest RMSE.

## Outputs:

• `permutation_importance_fold.csv` → RMSE increase per variable per fold.

• `Deception_2025_CO2_prediction.csv` → Original spatio-temporal and CO2 real data + new column 'CO2_predicted' from the best-performing fold.

### Notes:
• Predicted values are descaled (inverse of z-score) before saving.

• Best fold predictions are aligned with the original input data.

• The model is ready for technical validation and integration into data descriptor papers or repositories.

---

**Author:** Susana Flecha, Instituto de Ciencias Marinas de Andalucía-Consejo Superior de Investigaciones Científicas (CSIC)

**Last updated:** September 2025

**More info:** Flecha et al., 2025. Integrated meteocean and seismic dataset for AI-based seawater CO2 estimation at Deception Island, Antarctica. Scientific Data. and High-resolution meteocean and seismic dataset for AI modeling of seawater CO₂ at Deception Island-DIGITAL.CSIC doi: 


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
import shap
import matplotlib.pyplot as plt
import copy
%matplotlib inline


2025-09-11 17:41:44.062296: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
# Load data
df = pd.read_csv("Deception_2025_CO2_ocean_meteo_seismic.csv", sep=";")
df['Date'] = pd.to_datetime(df['Date'])

# Normalized temporal variables
df['hour'] = df['Date'].dt.hour / 23.0
df['month'] = (df['Date'].dt.month - 1) / 11.0
df['doy'] = df['Date'].dt.dayofyear / 365.0


/var/folders/dv/6kz620lx753268s1bl2zglg00000gn/T/ipykernel_34370/1697283840.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


In [4]:
# Align df for later use
features = ["Seawater_Temperature", "Salinity", "Latitude", "Longitude", "Solar_radiation",
            "Tidal_elevation", "Wind_speed", "Air_temperature", "Seismic_events", "Seismic_events_10_min_avg"]

target_var = "CO2"

# Eliminate NaNs
# df = df.dropna(subset=features + [target_var])

# CO2 lower than 400 filter (uncomment if desired)
# df = df[df[target_var] < 400]


In [5]:
# Create X, y, groups
X_raw = df[features].values
y_raw = df[[target_var]].values

# Create window withouth scaling
window_size = 10
X, y, groups = [], [], []

for i in range(len(X_raw) - window_size):
    X.append(X_raw[i:i + window_size])
    y.append(y_raw[i + window_size])
    groups.append(int(df.iloc[i + window_size]["Latitude"] * 500))

X = np.array(X)
y = np.array(y)
groups = np.array(groups)

# ✅ Align the original DataFrame with X, y, groups (due to windowing)
df_aligned = df.iloc[window_size:].reset_index(drop=True)


In [6]:
# Scale data
def create_model(input_shape):
    model = Sequential([
        Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001)), input_shape=input_shape),
        Dropout(0.3),
        Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001))),
        Dropout(0.3),
        Bidirectional(LSTM(32, kernel_regularizer=l2(0.001))),
        Dense(64, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0005),
                  loss='mse',
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model


In [7]:
# Cross-validation with GroupKFold

gkf = GroupKFold(n_splits=5)
val_rmse_list = []
all_y_true = []
all_y_pred = []
history_list = []
scalers_y = []  
val_indices_per_fold = []
best_result = None

for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
    print(f"\n📦 Fold {fold + 1}")

   # Dividing data
    X_train_raw, X_val_raw = X[train_idx], X[val_idx]
    y_train_raw, y_val_raw = y[train_idx], y[val_idx]

    # ⚠️ Fold scaling
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train = scaler_X.fit_transform(X_train_raw.reshape(-1, X.shape[2])).reshape(X_train_raw.shape)
    X_val   = scaler_X.transform(X_val_raw.reshape(-1, X.shape[2])).reshape(X_val_raw.shape)

    y_train = scaler_y.fit_transform(y_train_raw.reshape(-1, 1)).flatten()
    y_val   = scaler_y.transform(y_val_raw.reshape(-1, 1)).flatten()

    scalers_y.append(scaler_y)  # Guardar para invertir luego
    
    val_indices_per_fold.append(val_idx)

    # Model creation and training
    model = create_model((window_size, X.shape[2]))

    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=15, restore_best_weights=True
    )

    history = model.fit(
        X_train, y_train,
        epochs=200, batch_size=32,
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=1, shuffle=True
    )

    history_list.append(history.history)

    y_pred = model.predict(X_val)
    val_rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    val_rmse_list.append(val_rmse)
    all_y_true.append(y_val)
    all_y_pred.append(y_pred)


    print("📊 Calculating permutation importance...")

    # 🧪 Stablishing local seed for permutation
    np.random.seed(42 + fold)

    # Base prediction in real units
    y_val_inv = scaler_y.inverse_transform(y_val.reshape(-1, 1))
    y_pred_inv = scaler_y.inverse_transform(model.predict(X_val)).flatten()
    baseline_rmse = np.sqrt(mean_squared_error(y_val_inv, y_pred_inv))

    # Permutation importance per variable
    importance_fold = {}
    for i, var in enumerate(features):
        X_val_perm = np.array(X_val, copy=True)
        for t in range(window_size):
            X_val_perm[:, t, i] = np.random.permutation(X_val_perm[:, t, i])

        # Permutation importance in real units
        y_perm_pred = scaler_y.inverse_transform(model.predict(X_val_perm)).flatten()
        perm_rmse = np.sqrt(mean_squared_error(y_val_inv, y_perm_pred))
        importance_fold[var] = perm_rmse - baseline_rmse

    # 🔽 Save results CSV
    df_perm_fold = pd.DataFrame({
    "variable": list(importance_fold.keys()),
    "RMSE_increase": list(importance_fold.values())
    })
    csv_name = f"permutation_importance_fold{fold+1}.csv"
    df_perm_fold.to_csv(csv_name, index=False)
    print(f"✅ Saved: {csv_name}")

  
    last_val_idx = val_idx

    if best_result is None or val_rmse < min(val_rmse_list[:-1]):
        best_result = {
        "y_true": y_val,
        "y_pred": y_pred,
        "fold": fold,
        "val_idx": val_idx,
        "scaler_y": scaler_y  
        }

df_val = df_aligned.iloc[last_val_idx].copy()  # ✅ Correctly aligned DataFrame for the last fold


📦 Fold 1


2025-09-11 17:43:28.339658: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-09-11 17:43:28.340488: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-09-11 17:43:28.340990: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/200


2025-09-11 17:43:28.884975: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-09-11 17:43:28.886031: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-09-11 17:43:28.886709: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

68/69 [============================>.] - ETA: 0s - loss: 1.0442 - root_mean_squared_error: 0.6904

...

68/69 [============================>.] - ETA: 0s - loss: 1.0573 - root_mean_squared_error: 0.7007

2025-09-11 17:46:03.374565: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-09-11 17:46:03.375573: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-09-11 17:46:03.376244: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

69/69 [==============================] - 6s 22ms/step - loss: 1.0559 - root_mean_squared_error: 0.7001 - val_loss: 0.5772 - val_root_mean_squared_error: 0.2623
Epoch 2/200
69/69 [==============================] - 1s 9ms/step - loss: 0.7232 - root_mean_squared_error: 0.5085 - val_loss: 0.4509 - val_root_mean_squared_error: 0.1639
Epoch 3/200
69/69 [==============================] - 1s 8ms/step - loss: 0.4878 - root_mean_squared_error: 0.3146 - val_loss: 0.3631 - val_root_mean_squared_error: 0.1014
Epoch 4/200
69/69 [==============================] - 1s 8ms/step - loss: 0.3791 - root_mean_squared_error: 0.2387 - val_loss: 0.3067 - val_root_mean_squared_error: 0.1188
Epoch 5/200
69/69 [==============================] - 1s 8ms/step - loss: 0.3113 - root_mean_squared_error: 0.2095 - val_loss: 0.2752 - val_root_mean_squared_error: 0.1795
Epoch 6/200
69/69 [==============================] - 1s 8ms/step - loss: 0.2578 - root_mean_squared_error: 0.1864 - val_loss: 0.2204 - val_root_mean_squared

2025-09-11 17:46:53.988214: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-09-11 17:46:53.989089: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-09-11 17:46:53.989891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

18/18 [==============================] - 1s 3ms/step
📊 Calculating permutation importance...
18/18 [==============================] - 0s 2ms/step
✅ Saved: permutation_importance_fold5.csv


In [8]:
# Save predictions from the best fold aligned with the spatio-temporal input file
if best_result is not None:
    # Get the data from the best fold
    best_y_true = best_result["y_true"]
    best_y_pred = best_result["y_pred"]
    best_val_idx = best_result["val_idx"]
    best_scaler_y = best_result["scaler_y"]
    best_fold_num = best_result["fold"]

    # Invert scalation to obtain real values
    y_true_real = best_scaler_y.inverse_transform(best_y_true.reshape(-1, 1)).flatten()
    y_pred_real = best_scaler_y.inverse_transform(best_y_pred.reshape(-1, 1)).flatten()

    # Create DataFrame with date, coordinates, real CO2, and predicted CO2
    df_best_fold = pd.DataFrame({
        'Date': df_aligned.iloc[best_val_idx]['Date'].values,
        'Latitude': df_aligned.iloc[best_val_idx]['Latitude'].values,
        'Longitude': df_aligned.iloc[best_val_idx]['Longitude'].values,
        'CO2_real': y_true_real,
        'CO2_predicted': y_pred_real
    })

    # Save the file
    df_best_fold.to_csv("Deception_2025_CO2_prediction.csv", index=False)
    print("✅ Saved File with Best Fold Predictions (Date, Coordinates, Real CO2, Predicted CO2).")

✅ Saved File with Best Fold Predictions (Date, Coordinates, Real CO2, Predicted CO2).
